In [1]:
import asyncio
import json
import requests
from datetime import datetime, timezone

from app.utility.helps import Bob

In [2]:
from app.modules.activity import main as Activity
from app.modules.apps import main as Apps
from app.modules.catalog import main as Catalog
from app.modules.graph import main as Graph
from app.modules.tenant import main as Tenant
from app.modules.refreshhistory import main as RefreshHistory
from app.modules.refreshables import main as Refreshables
from app.modules.gateway import main as Gateway


In [3]:
bob = Bob()
state = bob.get_state()
settings = bob.get_settings()
headers = bob.get_context()

In [4]:
state

{'activity': {'lastRun': '2024-04-03T13:56:00.986056Z'},
 'catalog': {'lastFullScan': '2024-03-11T15:42:51.189058Z',
  'lastRun': '2024-04-04T13:56:00.986116Z'},
 'apps': {'lastRun': '2024-04-04T13:56:00.986056Z'},
 'graph': {'lastRun': '2024-04-04T13:56:00.986056Z'},
 'tenant': {'lastRun': '2024-04-04T13:56:00.986056Z'},
 'refreshhistory': {'lastRun': '2024-04-04T13:56:00.986056Z'},
 'refreshables': {'lastRun': '2024-04-05T13:56:00.986056Z'},
 'gateway': {'lastRun': '2024-04-04T13:56:00.986056Z'}}

In [5]:
from croniter import croniter

def scheduler(cron_time, last_run=None):

    year = datetime.now().year
    month = datetime.now().month
    day = datetime.now().day

    local_date = datetime(year, month, day, tzinfo=timezone.utc)
    val = croniter(cron_time, local_date).get_next(datetime)
    last_run = bob.convert_dt_str(last_run)
    print(last_run)

    # Determine if the current day of the week is the same as the value for day of the week in the cron value
    current_day_of_week = datetime.now().strftime("%A")
    cron_day_of_week = cron_time.split(" ")[4]


    # Get the next scheduled datetime
    next_datetime = val
    print(next_datetime)

    # Check if the current datetime matches the next scheduled datetime
    if datetime.now().strftime("%Y-%m-%d") == next_datetime.strftime("%Y-%m-%d"):
        print("Hello World, I am supposed to run today")
    else:
        print("Goodnight World, I am sleeping")


In [14]:
def is_function_due(cron_syntax, last_run):
    last_run_datetime = last_run

    cron = croniter(cron_syntax, last_run_datetime)
    next_run_datetime = cron.get_next(datetime)
    
    print(f"What is the next run date value {next_run_datetime} and what is the current datetime {datetime.now()}")

    if next_run_datetime.strftime("%Y-%m-%d") <= datetime.now().strftime("%Y-%m-%d"):
        return True
    else:
        return False



In [15]:
from datetime import datetime
from croniter import croniter
modules = settings.get("ApplicationModules").replace(" ","").split(",")
classes = [globals()[module] for module in modules]


for module in modules:
    cron = settings.get(f"{module}_cron")
    run = state.get(f"{module.lower()}").get("lastRun")

    last_run = bob.convert_dt_str(run)    

    

    if is_function_due(cron,last_run):
        print(f"Function {module} is due to run")
    else:
        print(f"Function {module} is not due to run")




What is the next run date value 2024-04-04 00:00:00 and what is the current datetime 2024-04-05 12:19:17.262328
Function Activity is due to run
What is the next run date value 2024-04-11 00:00:00 and what is the current datetime 2024-04-05 12:19:17.262661
Function Apps is not due to run
What is the next run date value 2024-04-06 00:00:00 and what is the current datetime 2024-04-05 12:19:17.262836
Function Catalog is not due to run
What is the next run date value 2024-04-06 00:00:00 and what is the current datetime 2024-04-05 12:19:17.263009
Function Graph is not due to run
What is the next run date value 2024-04-06 00:00:00 and what is the current datetime 2024-04-05 12:19:17.263175
Function Tenant is not due to run
What is the next run date value 2024-04-06 00:00:00 and what is the current datetime 2024-04-05 12:19:17.263428
Function RefreshHistory is not due to run
What is the next run date value 2024-04-05 00:00:00 and what is the current datetime 2024-04-05 12:19:17.263645
Function

In [ ]:
settings.get("Refreshables_cron")
state.get("refreshables").get("lastRun")

In [ ]:
state.get('activity').get('lastRun')

In [ ]:
api_url = 'https://api.powerbi.com/v1.0/myorg/gateways'
response = requests.get(url=api_url, headers=headers)
results = response.json()
print(response.status_code)
gateways = list()
for gateway in results['value']:
    gateways.append(gateway.get("id"))


In [ ]:
r = list()
for id in gateways:
    response = requests.get(f'https://api.powerbi.com/v1.0/myorg/gateways/{id}/datasources', headers=headers)
    doc_results = response.json()
    r.append(doc_results['value'])   


In [ ]:
gateways = list()

for i in range(0, len(r)):
    if isinstance(r[i], list):
        for j in range(0, len(r[i])):
            gateways.append(r[i][j])
    else:
        gateways.append(r[i])

In [ ]:
gateways

In [ ]:
users = list()

for i in range(0,len(gateways)):
    api_users = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/users'
    response = requests.get(api_users, headers=headers)
    results = response.json()
    results['value'][0]['datasourceId'] = gateways[i]["id"]
    results['value'][0]['gatewayId'] = gateways[i]["gatewayId"]
    users.append(results['value'][0])
    

## Datasource Connectivity  

This only reports back connectivity to the datasource as an error. Also, you need to use the response.text as the JSON is not populated. A successful connection will report back as HTTP 200 while a failed connection will result in HTTP 400

In [ ]:
## This will be used for a template dictionary for a successful connection to the datasource
status_base = {   
    "datasourceId":"",
    "gatewayId":"",
    "error":
    {
        "code":"success",
        "pbi.error":{ 
            "code":"0",
            "parameters":{},
            "details":[],
            "exceptionCulprit":0
        }
    }
}


In [ ]:
status  = list()
for i in range(0,len(gateways)):
    api_status = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/status'
    
    response = requests.get(api_status, headers=headers)
    if response.ok:
        # response.json() is actually empty when it is a success
        
        status_base['datasourceId'] = gateways[i]["id"]
        status_base['gatewayId'] = gateways[i]["gatewayId"]
        status_base.get("error").get("pbi.error").get("details").append({"message":"success","detail":response.status_code})    
        status.append(status_base)
    else:
        results = json.loads(response.text)
        results["datasourceId"] = gateways[i]["id"]
        results["gatewayId"] = gateways[i]["gatewayId"]
        results.get("error").get("pbi.error").get("details").append({"message":"cannot connect","detail":response.status_code})    
        status.append(results)


In [ ]:
datasources = list()

for i in range(0,len(gateways)):
    api_datasource = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}'
    response = requests.get(api_datasource, headers=headers)
    results = response.json()
    results.pop('@odata.context')
    datasources.append(results)


In [ ]:
import smtplib
from email.mime.text import MIMEText

sender_email = 'brcampb@microsoft.com'
receiver_email = 'brandonh.campbell@gmail.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Gmail)
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())



sender_email = 'your_email@example.com'
receiver_email = 'recipient_email@example.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Outlook)
with smtplib.SMTP('smtp.office365.com', 587) as server:
    server.starttls()
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())


In [ ]:
import smtplib
from email.mime.text import MIMEText

# Set up the SMTP server
smtp_server = 'smtp.gmail.com'
smtp_port = 587
sender_email = 'your_email@example.com'
password = 'your_password'

# Create a message
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'
msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = 'recipient_email@example.com'

# Send the email
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, [msg['To']], msg.as_string())


In [ ]:
# rest_api = "admin/capacities/refreshables"
# GET https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
# htpps://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
rest_api = "https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets&$top=5000"

response = requests.get(url=rest_api, headers=headers)
workspaces = response.json()


In [ ]:
workspaces

In [ ]:
for item in workspaces['value']:
    group_id = item['id']
    for dataset in item['datasets']:
        dataset_id = dataset['id']
        print(dataset)

In [ ]:
# GET https://api.powerbi.com/v1.0/myorg/groups/{groupId}/datasets/{datasetId}/refreshes
refresh_history = list()

for item in workspaces['value']:
    group_id = item['id']
    for dataset in item['datasets']:
        dataset_id = dataset['id']
        if dataset['isRefreshable']==True and dataset['addRowsAPIEnabled']==False:
            rest_api = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/datasets/{dataset_id}/refreshes"

            response = requests.get(url=rest_api, headers=headers)
            if response.ok:
                results = response.json()
                for result in results['value']:
                    if len(result)>0:
                        refresh_history.append(result)




In [ ]:
refresh_history

In [ ]:
import pandas as pd

fd = pd.DataFrame(refresh_history)
fd.head()

In [ ]:
fd[fd['requestId']=='9cbca636-ec64-4ff6-9dfe-9e2e5acc3056']['refreshAttempts'][0]

In [ ]:
import pandas as pd

def normalize(df:pd.DataFrame(), base2:pd.DataFrame(), json_column='workspace_id'):
    for row in df.itertuples():
        column_number = df.columns.get_loc(json_column)+1
        if isinstance(row[column_number], list) and len(row[column_number])>0:
            for item in row[column_number]:
                # item[id_name] = row.id
                base2 = pd.concat([base2, df, pd.json_normalize(item)])
                base2.drop(json_column, axis=1, inplace=True)
        else:
            if isinstance(row[column_number], dict):
                print("Yes it is a dictionary")
                base2 = pd.concat([base2, df, pd.json_normalize(row[column_number])])
                base2.drop(json_column, axis=1, inplace=True)
    return base2

In [ ]:
dd = normalize(fd, pd.DataFrame(), 'refreshAttempts')
dd.head()

In [ ]:
de = normalize(dd, pd.DataFrame(), 'serviceExceptionJson')
de

In [ ]:
import json

string_dict = "{'id': '8271b9f0-3ff7-46b5-9f8a-5a3ba59d01b7', 'name': 'FabricMonitor', 'addRowsAPIEnabled': False, 'configuredBy': 'brandon.campbell@mngenvmcap084084.onmicrosoft.com', 'isRefreshable': True, 'isEffectiveIdentityRequired': False, 'isEffectiveIdentityRolesRequired': False, 'targetStorageMode': 'Abf', 'createdDate': '2024-03-27T18:42:49.663Z', 'contentProviderType': 'PbixInImportMode', 'upstreamDatasets': [], 'users': [], 'isInPlaceSharingEnabled': False}"



print(dict_obj)


In [ ]:
for row in dd.itertuples():
    if isinstance(row.serviceExceptionJson, list) and len(row.serviceExceptionJson)>0:
        for item in row.serviceExceptionJson:
            print(item)
    else:
        if isinstance(row.serviceExceptionJson, str):
            print("Yes it is a dictionary")
            print(row.serviceExceptionJson)

In [ ]:
api="https://api.powerbi.com/v1.0/myorg/admin/capacities/refreshables?$top=5000"

refreshables = list()

response = requests.get(url=api, headers=headers)
if response.ok:
    results = response.json()
    for result in results['value']:
        refreshables.append(result)

        

In [ ]:
if len(refresh_history)>=len(refreshables):
    print("there is more information in refresh_history than in refreshables")
else:
    print("there is more information in refreshables than in refresh_history")

In [ ]:
refresh_history[0]

In [ ]:
import pandas as pd

df = pd.json_normalize(refreshables)
df.head()

In [ ]:
df2 = pd.json_normalize(refresh_history)

In [ ]:
df2.head()

In [ ]:
df.to_csv('refreshables.csv', index=False)
df2.to_csv('refresh_history.csv', index=False)

In [ ]:
df3 = df2[['requestId','refreshAttempts']]

In [ ]:
import pandas as pd

# Assuming the DataFrame with the column 'refreshAttempts' is named 'df'
df_expanded = pd.json_normalize(df3['refreshAttempts'])

# Merge the expanded DataFrame with the original DataFrame
df_merged = pd.concat([df3.drop('refreshAttempts', axis=1), df_expanded], axis=1)


In [ ]:
df_merged.head()

In [ ]:
df_unpivoted = df_merged.melt(id_vars=['requestId'], var_name='column', value_name='value')
df_unpivoted.rename(columns={'requestId':'requestId','column':'column','value':'refreshAttempts'}, inplace=True)



In [ ]:
df_unpivoted.head()

In [ ]:
import pandas as pd
import json

# Convert JSON column to DataFrame
df_refreshAttempts = pd.json_normalize(df_unpivoted['refreshAttempts'])

# Merge the original DataFrame with the new DataFrame
df_merged = pd.concat([df_unpivoted.drop('refreshAttempts', axis=1), df_refreshAttempts], axis=1)
df_merged.head()

In [ ]:
refresh_history_clean = df_merged[['requestId','attemptId','startTime','endTime','serviceExceptionJson']]
refresh_history_clean.head()

In [ ]:
refresh_history_cleanest = df2.merge(refresh_history_clean, on='requestId', how='left')
refresh_history_cleanest.head()

In [ ]:
refresh_history_cleanest.drop(columns=['refreshAttempts'], inplace=True)

In [ ]:
refresh_history_cleanest.to_csv('refresh_history_cleanest.csv', index=False)